In [1]:
import pandas as pd

In [48]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')
bike_realtime_df.drop(columns="create_time",inplace=True)

In [49]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["month"] = bike_realtime_df["source_time"].dt.month
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [50]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [51]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].min()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].min()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [52]:
mrt_station_table = youbike_mrt_distance_df.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500107102,O02,6.986596,景安
1,500107099,O02,7.208812,景安
2,500107065,O02,6.809924,景安
3,500107086,O02,11.338723,景安
4,500107045,O02,6.757624,景安
...,...,...,...,...
171210,500119045,O54,10.703496,蘆洲
171211,500119070,O54,11.054859,蘆洲
171212,500119048,O54,10.971054,蘆洲
171213,500119077,O54,10.791606,蘆洲


In [53]:
df  = df_bike_hour_grby.merge(bike_station_df.loc[:,["bike_station_id","total_space","lat","lng"]],
                              how="left",
                              on="bike_station_id")

In [54]:
df = df.merge(time_table_df.loc[:,["date","day_of_week","day_of_week_name"]],
            on="date",
            how="left")

In [55]:
idx = youbike_mrt_distance_df.groupby("bike_station_id")["distance"].idxmin()
youbike_min_distance_table = youbike_mrt_distance_df.loc[idx,].reset_index(drop=True)

In [56]:
mrt_station_table = youbike_min_distance_table.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500101001,BR08,0.015941,科技大樓
1,500101002,BR08,0.053192,科技大樓
2,500101003,BR08,0.302305,科技大樓
3,500101004,BR08,0.296290,科技大樓
4,500101005,BR08,0.510676,科技大樓
...,...,...,...,...
1410,500119087,G07,0.645005,公館
1411,500119088,BR08,0.683471,科技大樓
1412,500119089,G07,0.884494,公館
1413,500119090,G07,0.719453,公館


In [57]:
df = df.merge(mrt_station_table,on="bike_station_id",how="left")
df.head()

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name
0,500101001,2024-04-16,22,4,5,28,25.02605,121.5436,3,Tuesday,BR08,0.015941,科技大樓
1,500101001,2024-04-16,23,22,4,28,25.02605,121.5436,3,Tuesday,BR08,0.015941,科技大樓
2,500101001,2024-04-17,0,16,4,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓
3,500101001,2024-04-17,1,16,11,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓
4,500101001,2024-04-17,2,18,10,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓


In [58]:
mrt_history_df = mrt_history_df.merge(time_table_df.loc[:,["date","day_of_week",]],
                                        on="date",
                                        how="left")

In [59]:
mrt_history_df_group_byweek_enter = pd.DataFrame(mrt_history_df.groupby(["mrt_station","day_of_week","hour"])["enter_num"].min()).reset_index(drop=False)
mrt_history_df_group_byweek_exit = pd.DataFrame(mrt_history_df.groupby(["mrt_station","day_of_week","hour"])["exit_num"].min()).reset_index(drop=False)

In [60]:
mrt_history_df_group_byweek = mrt_history_df_group_byweek_enter.merge(
    mrt_history_df_group_byweek_exit,
    left_on = ["mrt_station","day_of_week","hour"],
    right_on = ["mrt_station","day_of_week","hour"],
    how = "left"
)
mrt_history_df_group_byweek

,mrt_station,day_of_week,hour,enter_num,exit_num
0,七張,1,0,24,102.0
1,七張,1,1,0,2.0
2,七張,1,5,0,0.0
3,七張,1,6,109,62.0
4,七張,1,7,247,122.0
...,...,...,...,...,...
17341,龍山寺,7,19,1367,1383.0
17342,龍山寺,7,20,1178,1307.0
17343,龍山寺,7,21,987,1287.0
17344,龍山寺,7,22,762,1120.0


In [61]:
df = df.merge(
    mrt_history_df_group_byweek.loc[:,["day_of_week","hour","mrt_station","enter_num",	"exit_num"]],
    left_on=["day_of_week","hour",	"station_name"],
    right_on=["day_of_week","hour", "mrt_station"],
    how="left"
)

In [62]:
def if_not_bike(row):
    if row["aval_bike_mean"]<=5 or row["aval_bike_mean"]<=row["total_space"]*0.1:
        return(1)
    else:
        return(0)
df["y"] = df.apply(if_not_bike,axis=1)

In [63]:
df_count_y = pd.DataFrame(df.groupby(["bike_station_id","day_of_week","hour"])["y"].count()).reset_index(drop=False)
df_sum_y = pd.DataFrame(df.groupby(["bike_station_id","day_of_week","hour"])["y"].sum()).reset_index(drop=False)
df_if_not_bike_grby =  df_count_y.merge(df_sum_y,left_on=["bike_station_id","day_of_week","hour"],right_on=["bike_station_id","day_of_week","hour"],how="left",suffixes=["_count","_sum"])
df_if_not_bike_grby 

,bike_station_id,day_of_week,hour,y_count,y_sum
0,500101001,1,0,5,5
1,500101001,1,1,5,5
2,500101001,1,2,5,5
3,500101001,1,3,5,5
4,500101001,1,4,5,5
...,...,...,...,...,...
237793,500119091,7,19,5,4
237794,500119091,7,20,5,4
237795,500119091,7,21,5,3
237796,500119091,7,22,5,3


In [64]:
dfcom = df_if_not_bike_grby.merge(df.loc[:,["bike_station_id","day_of_week","hour","mrt_station_id","distance","station_name","enter_num","exit_num"]],
                          left_on=["bike_station_id","day_of_week","hour"],
                          right_on=["bike_station_id","day_of_week","hour"],
                          how="left")

In [65]:
dfcom.drop_duplicates(inplace=True)

In [66]:
dfcom.to_csv("test4.csv",index=False,encoding="utf-8-sig")

In [1]:
dfcom

NameError: name 'dfcom' is not defined